In [8]:
#------------------------- IMPORT LIBRARIES -------------------------#
import torchvision, torch
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os
import logging
import torch
import torch.nn as nn
import torch.optim as optim
from torch.backends import cudnn
import torchvision
from torchvision import transforms
from torchvision.models import alexnet

# FILE DOVE METTIAMO TUTTE LE FUNZIONI CHE CI SERVONO, PER NON INTASARE IL MAIN
from utils import *
from PIL import Image

In [9]:
# Uso = [1,8] o range(inizio, fine+1)
# OGNI CLASSE HA 500 IMMAGINI: 250 TRAIN + 250 VALIDATION
# SE VOGLIO UNA SOLO CLASSE NON USARE RANGE() MA mettere solo il numero classes=5
train_dataloader, val_dataloader = get_train_val_dataloader(classes=range(0, 100))

Files already downloaded and verified
Train Dataset: 25000
Valid Dataset: 25000


In [10]:
# Ogni classe ha 100 immagini
test_dataloader  = get_test_dataloader(classes=range(0,100))

Files already downloaded and verified
Test Dataset: 10000


In [18]:
#--------------------------------------------------
# Implemento ResNet18 per il 1° punto del progetto
#--------------------------------------------------
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', pretrained=False)
# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# I need to change this because we have NUM_CLASSES
##net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES) 

# We just changed the last layer of AlexNet with a new fully connected layer with 100 outputs
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
parameters_to_optimize = model.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

Using cache found in C:\Users\nicoa/.cache\torch\hub\pytorch_vision_v0.6.0


Linear(in_features=512, out_features=1000, bias=True)


Linear(in_features=512, out_features=100, bias=True)

In [20]:
model = model.to(DEVICE)
cudnn.benchmark # Calling this optimizes runtime
model, train_acc_history, train_loss_history, val_acc_history, val_loss_history = train_model(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler)

False

In [ ]:
# VEDERE COME MAI SE AUMENTO IL NUMERO DI EPOCH LA LOSS INIZIALE AUMENTA
plt.figure(figsize=(20, 10))
plt.plot(range(NUM_EPOCHS), val_loss_history, marker="s", color="blue")
plt.plot(range(NUM_EPOCHS), train_loss_history, marker="s", color="green")
plt.xlabel("Epoch")
plt.ylabel("loss")
plt.xticks(range(0, NUM_EPOCHS, 1))
plt.legend(["validation", "train"])
plt.show()

In [ ]:
# VEDERE COME MAI SE AUMENTO IL NUMERO DI EPOCH LA LOSS INIZIALE AUMENTA
plt.figure(figsize=(20, 10))
plt.plot(range(NUM_EPOCHS), val_acc_history, marker="s", color="blue")
plt.plot(range(NUM_EPOCHS), train_acc_history, marker="s", color="green")
plt.xticks(range(0, NUM_EPOCHS, 1))
plt.yticks(np.arange(0, 1.2, 0.2))
plt.xlabel("Epoch")
plt.ylabel("accuracy")
plt.legend(["validation", "train"])
plt.show()

In [ ]:
model = model.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
model.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in test_dataloader:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))
print('Test Accuracy: {}'.format(accuracy))

In [19]:
for i in range(0, 100, 10):
    train_dataloader, val_dataloader = get_train_val_dataloader(classes=range(i, i+10))
    # MODIFICARE ULTIMO LIVELLO PER AGGIUNGERE 10 CLASSI AL FC-CLASSIFIER

Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
Files already downloaded and verified
Train Dataset: 2500
Valid Dataset: 2500
